In [ ]:
#imports

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#contains synergy values and fold split (numbers 0-4)
labels_file_path = "/home/nidhi/Documents/freelancing/DeepSynergy/data/labels.csv"
labels = pd.read_csv( labels_file_path, index_col=0)

#labels are duplicated for the two different ways of ordering in the data
labels = pd.concat([labels, labels]) 

print(labels.head())
print(labels.shape)

In [ ]:
#grouped by cell_line. This will be used for density plots.
grouped = labels.groupby('cell_line')

for name,group in grouped:
   print(name)
   #print(group.shape)

KDE plot

In [ ]:
A2058_data = grouped.get_group("A2058")
A2058_data.head()

Plot with bandwidth = 3.589

In [ ]:
# plot prediction and density plot together
#fake prediction variable.
fake_pred = 1.5  
bandwidth = 3.589
cell_line_name = "A2058"

graph = sns.kdeplot(A2058_data["synergy"], bw_adjust= 3.589)
graph.axvline(fake_pred)
plt.xlabel("N = {0}      Bandwidth = {1}".format(A2058_data.shape[0], bandwidth))
plt.title("train data distribution of {0}".format(cell_line_name))
#plt.savefig("KDE_plot_fixed_bandwidth.png")
plt.show()


KDE Plot with bandwidth calculated by Scipy and Statsmodels.

In [ ]:
# plot prediction and density plot together
#fake prediction variable.
fake_pred = 1.5  
bandwidth = 3.589
cell_line_name = "A2058"

graph = sns.kdeplot(A2058_data["synergy"])
graph.axvline(fake_pred)
plt.xlabel("Synergy Score")
plt.title("train data distribution of {0}".format(cell_line_name))
#plt.savefig("Training_data_density_plot.png")
plt.show()


Working on Heatmap now.

In [ ]:
grouped_drugs = labels.groupby(["drug_a_name", "drug_b_name", "cell_line"])

In [ ]:
#heatmap can't be created with duplicate entries.
#remove duplicate entries by taking synergy mean values for duplicates.

deduplicated_labels = grouped_drugs["synergy"].mean().reset_index(name = "synergy_mean")
#print(deduplicated_labels.nunique)
print(deduplicated_labels.head())

In [ ]:
#group deduplicated_labels by cell_line to generate heatmaps for cell_line individually.
#deduplicated_labels = ddl

ddl_grouped = deduplicated_labels.groupby("cell_line")
ddl_grouped_keys = [key for key, _ in ddl_grouped]

print(len(ddl_grouped_keys))


HeatMap for A2058

In [ ]:
#ready data

ddl_A2058 = ddl_grouped.get_group("A2058")
#print(ddl_A2058.head())
print(ddl_A2058.shape)

#print(ddl_A2058.iloc[:,[0,1]].nunique())
#double check drug pairs are unique.
unique_drugpairs_in_ddl_2058 = [key for key, _ in ddl_A2058.groupby(["drug_a_name","drug_b_name"])]
print(len(unique_drugpairs_in_ddl_2058))

#remove cell_line column. 
A2058_heatmap = ddl_A2058.iloc[:,[0,1,3]].copy()
A2058_heatmap = ddl_A2058.pivot("drug_a_name", "drug_b_name", "synergy_mean")


In [ ]:

#sns.color_palette("RdBu")
sns.heatmap(A2058_heatmap, center = 0, vmin = -150, vmax = 150, cmap = "RdBu")
plt.savefig("Heatmap.png")
plt.show()
